# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 27 2022 2:47PM,238305,10,0085720429,ISDIN Corporation,Released
1,May 27 2022 2:47PM,238305,10,0085720431,ISDIN Corporation,Released
2,May 27 2022 2:47PM,238305,10,0085720430,ISDIN Corporation,Released
3,May 27 2022 2:47PM,238305,10,0085720428,ISDIN Corporation,Released
4,May 27 2022 2:47PM,238305,10,0085720432,ISDIN Corporation,Released
5,May 27 2022 2:47PM,238305,10,0085720434,ISDIN Corporation,Released
6,May 27 2022 2:47PM,238305,10,0085720435,ISDIN Corporation,Released
7,May 27 2022 2:47PM,238305,10,0085720436,ISDIN Corporation,Released
8,May 27 2022 2:47PM,238305,10,0085720433,ISDIN Corporation,Released
9,May 27 2022 2:47PM,238305,10,0085720456,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,238302,Released,2
27,238303,Released,1
28,238304,Released,1
29,238305,Released,26
30,238306,Released,18


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
238302,NaN,NaN,NaN,2.0
238303,NaN,NaN,NaN,1.0
238304,NaN,NaN,NaN,1.0
238305,NaN,NaN,NaN,26.0
238306,NaN,NaN,NaN,18.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
238200,1.0,0.0,0.0,0.0
238212,0.0,0.0,1.0,1.0
238214,0.0,0.0,0.0,1.0
238219,0.0,0.0,0.0,1.0
238225,0.0,0.0,8.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
238200,1.0,0,0,0
238212,0.0,0,1,1
238214,0.0,0,0,1
238219,0.0,0,0,1
238225,0.0,0,8,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,238200,1.0,0,0,0
1,238212,0.0,0,1,1
2,238214,0.0,0,0,1
3,238219,0.0,0,0,1
4,238225,0.0,0,8,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,238200,1.0,,,
1,238212,0.0,,1,1
2,238214,0.0,,,1
3,238219,0.0,,,1
4,238225,0.0,,8,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 27 2022 2:47PM,238305,10,ISDIN Corporation
26,May 27 2022 2:47PM,238301,10,ISDIN Corporation
55,May 27 2022 2:45PM,238306,10,ISDIN Corporation
73,May 27 2022 2:40PM,238304,16,Sartorius Bioprocess Solutions
74,May 27 2022 2:39PM,238303,16,Sartorius Bioprocess Solutions
75,May 27 2022 2:35PM,238302,10,"Citieffe, Inc."
77,May 27 2022 2:05PM,238300,19,ACG North America LLC
78,May 27 2022 1:53PM,238299,16,American International Chemical
79,May 27 2022 1:31PM,238297,19,Eli Lilly and Company
80,May 27 2022 1:14PM,238289,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,May 27 2022 2:47PM,238305,10,ISDIN Corporation,0.0,,,26
1,May 27 2022 2:47PM,238301,10,ISDIN Corporation,0.0,,,29
2,May 27 2022 2:45PM,238306,10,ISDIN Corporation,0.0,,,18
3,May 27 2022 2:40PM,238304,16,Sartorius Bioprocess Solutions,0.0,,,1
4,May 27 2022 2:39PM,238303,16,Sartorius Bioprocess Solutions,0.0,,,1
5,May 27 2022 2:35PM,238302,10,"Citieffe, Inc.",0.0,,,2
6,May 27 2022 2:05PM,238300,19,ACG North America LLC,0.0,,,1
7,May 27 2022 1:53PM,238299,16,American International Chemical,0.0,,,1
8,May 27 2022 1:31PM,238297,19,Eli Lilly and Company,0.0,,,1
9,May 27 2022 1:14PM,238289,10,ISDIN Corporation,0.0,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 27 2022 2:47PM,238305,10,ISDIN Corporation,26,,
1,May 27 2022 2:47PM,238301,10,ISDIN Corporation,29,,
2,May 27 2022 2:45PM,238306,10,ISDIN Corporation,18,,
3,May 27 2022 2:40PM,238304,16,Sartorius Bioprocess Solutions,1,,
4,May 27 2022 2:39PM,238303,16,Sartorius Bioprocess Solutions,1,,
5,May 27 2022 2:35PM,238302,10,"Citieffe, Inc.",2,,
6,May 27 2022 2:05PM,238300,19,ACG North America LLC,1,,
7,May 27 2022 1:53PM,238299,16,American International Chemical,1,,
8,May 27 2022 1:31PM,238297,19,Eli Lilly and Company,1,,
9,May 27 2022 1:14PM,238289,10,ISDIN Corporation,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 27 2022 2:47PM,238305,10,ISDIN Corporation,26,,
1,May 27 2022 2:47PM,238301,10,ISDIN Corporation,29,,
2,May 27 2022 2:45PM,238306,10,ISDIN Corporation,18,,
3,May 27 2022 2:40PM,238304,16,Sartorius Bioprocess Solutions,1,,
4,May 27 2022 2:39PM,238303,16,Sartorius Bioprocess Solutions,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 27 2022 2:47PM,238305,10,ISDIN Corporation,26.0,NaN,NaN
1,May 27 2022 2:47PM,238301,10,ISDIN Corporation,29.0,NaN,NaN
2,May 27 2022 2:45PM,238306,10,ISDIN Corporation,18.0,NaN,NaN
3,May 27 2022 2:40PM,238304,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
4,May 27 2022 2:39PM,238303,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 27 2022 2:47PM,238305,10,ISDIN Corporation,26.0,0.0,0.0
1,May 27 2022 2:47PM,238301,10,ISDIN Corporation,29.0,0.0,0.0
2,May 27 2022 2:45PM,238306,10,ISDIN Corporation,18.0,0.0,0.0
3,May 27 2022 2:40PM,238304,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
4,May 27 2022 2:39PM,238303,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2620996,90.0,21.0,4.0
15,238279,1.0,4.0,0.0
16,1667832,5.0,9.0,0.0
17,238256,1.0,0.0,0.0
19,476597,2.0,0.0,0.0
20,476531,16.0,18.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2620996,90.0,21.0,4.0
1,15,238279,1.0,4.0,0.0
2,16,1667832,5.0,9.0,0.0
3,17,238256,1.0,0.0,0.0
4,19,476597,2.0,0.0,0.0
5,20,476531,16.0,18.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,90.0,21.0,4.0
1,15,1.0,4.0,0.0
2,16,5.0,9.0,0.0
3,17,1.0,0.0,0.0
4,19,2.0,0.0,0.0
5,20,16.0,18.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,90.0
1,15,Released,1.0
2,16,Released,5.0
3,17,Released,1.0
4,19,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,17,19,20
Status,,,,,,
Completed,4.0,0.0,0.0,0.0,0.0,0.0
Executing,21.0,4.0,9.0,0.0,0.0,18.0
Released,90.0,1.0,5.0,1.0,2.0,16.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,17,19,20
0,Completed,4.0,0.0,0.0,0.0,0.0,0.0
1,Executing,21.0,4.0,9.0,0.0,0.0,18.0
2,Released,90.0,1.0,5.0,1.0,2.0,16.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,17,19,20
0,Completed,4.0,0.0,0.0,0.0,0.0,0.0
1,Executing,21.0,4.0,9.0,0.0,0.0,18.0
2,Released,90.0,1.0,5.0,1.0,2.0,16.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()